# 📊 K-Fold 교차 검증 - 5-Fold Cross Validation
> PRD 계획에 따른 K-Fold 교차 검증으로 모델 안정성 평가

**목표 성능**: ROUGE-F1 72-75

In [1]:
# 환경 설정
import sys
import os
from pathlib import Path

# 프로젝트 루트 경로 추가
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent.parent  # 3번만 parent 사용!

# 다른 프로젝트 경로 제거하고 현재 프로젝트 경로만 추가
sys.path = [p for p in sys.path if 'computer-vision-competition' not in p]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project Root: {project_root}")
print(f"Current Dir: {notebook_dir}")

# 필요한 라이브러리 임포트
import yaml
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import torch
from datetime import datetime
import wandb

# 커스텀 모듈 임포트
from src.logging.notebook_logger import NotebookLogger
from src.utils.gpu_optimization.team_gpu_check import check_gpu_tier

print("Libraries imported successfully!")

Project Root: /home/ieyeppo/AI_Lab/natural-language-processing-competition
Current Dir: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH
Libraries imported successfully!


In [2]:
# 설정 파일 로드
config_path = notebook_dir / 'configs' / 'config_kfold.yaml'

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print(f"K-Fold Splits: {config['kfold']['n_splits']}")
print(f"Model: {config['model']['name']}")
print(f"Ensemble Method: {config['kfold']['ensemble_method']}")
print(f"Save Each Fold: {config['kfold']['save_each_fold']}")

K-Fold Splits: 5
Model: upstage/SOLAR-10.7B-Instruct-v1.0
Ensemble Method: weighted_average
Save Each Fold: True


In [3]:
# 로그 디렉토리 생성
# config의 로그 경로 사용
def get_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

# config에 log_dir이 정의되어 있으면 사용, 없으면 기본값
if 'log_dir' in config['paths']:
    log_dir = get_path(config['paths']['log_dir'])
else:
    # 기본값: notebook_dir/logs/kfold
    log_dir = notebook_dir / 'logs' / 'kfold'

log_dir.mkdir(parents=True, exist_ok=True)

# 타임스탬프 생성
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 로거 초기화
log_file = log_dir / f'kfold_{config["kfold"]["n_splits"]}fold_{timestamp}.log'
logger = NotebookLogger(
    log_path=str(log_file),
    print_also=True
)

logger.write('='*50)
logger.write('K-Fold Cross Validation Experiment')
logger.write(f'Timestamp: {timestamp}')
logger.write(f'Folds: {config["kfold"]["n_splits"]}')
logger.write('='*50)

K-Fold Cross Validation Experiment
Timestamp: 20251010_090402
Folds: 5


In [4]:
# K-Fold 설정
kfold = KFold(
    n_splits=config['kfold']['n_splits'],
    shuffle=config['kfold']['shuffle'],
    random_state=config['kfold']['random_state']
)

logger.write(f"KFold configured:")
logger.write(f"  - Splits: {config['kfold']['n_splits']}")
logger.write(f"  - Shuffle: {config['kfold']['shuffle']}")
logger.write(f"  - Random State: {config['kfold']['random_state']}")

KFold configured:
  - Splits: 5
  - Shuffle: True
  - Random State: 42


In [5]:
# 데이터 로드
# config 파일의 경로 사용
def get_data_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

# config에서 데이터 경로 가져오기
train_path = get_data_path(config['paths']['train_file'])

logger.write(f"Loading data from config path:")
logger.write(f"  - Train: {train_path}")

# 데이터 로드
train_df = pd.read_csv(train_path)

logger.write(f"\nData loaded: {len(train_df)} samples")

# Fold별 데이터 분할 확인
for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(train_df), 1):
    logger.write(f"Fold {fold_idx}: Train={len(train_idx)}, Val={len(val_idx)}")
    print(f"Fold {fold_idx}: Train={len(train_idx)}, Val={len(val_idx)}")

Loading data from config path:
  - Train: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/train.csv

Data loaded: 12457 samples
Fold 1: Train=9965, Val=2492
Fold 1: Train=9965, Val=2492
Fold 2: Train=9965, Val=2492
Fold 2: Train=9965, Val=2492
Fold 3: Train=9966, Val=2491
Fold 3: Train=9966, Val=2491
Fold 4: Train=9966, Val=2491
Fold 4: Train=9966, Val=2491
Fold 5: Train=9966, Val=2491
Fold 5: Train=9966, Val=2491


In [6]:
# GPU 체크
if torch.cuda.is_available():
    gpu_tier = check_gpu_tier()
    logger.write(f"\nGPU: {torch.cuda.get_device_name(0)}")
    logger.write(f"GPU Tier: {gpu_tier}")
    
    if config['gpu']['empty_cache_between_folds']:
        logger.write("Will clear GPU cache between folds")


GPU: NVIDIA GeForce RTX 4090
GPU Tier: LOW
Will clear GPU cache between folds


In [7]:
# Fold별 성능 추적 초기화
fold_results = []

logger.write("\nStarting K-Fold Cross Validation...")
logger.write("-" * 50)


Starting K-Fold Cross Validation...
--------------------------------------------------


## K-Fold 학습 루프

각 Fold별로 모델을 학습하고 평가합니다.

In [8]:
# 예시: Fold 1 학습
fold_idx = 1
logger.write(f"\n=== Training Fold {fold_idx}/{config['kfold']['n_splits']} ===")

# GPU 캐시 정리 (설정된 경우)
if config['gpu']['empty_cache_between_folds'] and torch.cuda.is_available():
    torch.cuda.empty_cache()
    logger.write("GPU cache cleared")

# 실제 학습 코드는 config 설정에 따라 구현
logger.write(f"Fold {fold_idx} training would start here...")


=== Training Fold 1/5 ===
GPU cache cleared
Fold 1 training would start here...


## 앙상블 및 최종 평가

모든 Fold의 결과를 앙상블하여 최종 성능을 평가합니다.